# LlamaIndex と Gemini, ChramaDB で RAG を作ってみる

### RAG
1. あらかじめ用意したテキスト(群)を数値化(ベクター化/embedding)し、ベクトルDBに保存
2. テキストに対する質問文を数値化し、数値化されたベクトルDB中の各文書との距離(数値的差)を求め、距離の小さいものの上位を候補として抽出
3. 抽出されたテキストを、質問文とともにLLMへ投げると、質問文に合わせてテキストを解釈し、回答

pip install faiss-cpu  
pip install chromadb  

pip install llama-index   
pip install llama-index-embeddings-google-genai llama-index-llms-google-genai # for Gemini  
pip install llama-index-vector-stores-faiss                                   # for FAISS   
pip install llama-index-vector-stores-chroma                                  # for Chroma  

In [1]:
# Gemini API Keyを取得

# api_key = os.getenv("GOOGLE_API_KEY")     # 環境変数から取得（推奨）

with open('GOOGLE_API_KEY.txt', 'r') as f:  # ファイルからAPI Keyを取得
    api_key = f.read().strip()

# Geminiモデルを指定
llm_model       = 'gemini-2.0-flash-exp'
embedding_model = 'gemini-embedding-001'

## 簡単なLlamaIndex+Gemini+ChramaDBを利用した、RAGの例

In [2]:
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI

# 入力テキスト群
texts = [
    '石川です。エンジニアです。散歩は嫌いではありません。', 
    '果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。',
    '佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。',
    '東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。',
    'サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。',
    '伊藤さんと山本さんは、よく一緒に奥多摩へキャンプに行くようです',
    '中村さんは高橋さんの勤めている会社の上司です。よく飲みに誘われますが参加するかは半々です。',
    '京都にお住いの小林さんは、佐藤さんのおいです。',
    '加藤さんは5人家族です。',
]

# Document に変換
docs = [Document(text=t) for t in texts]
print(docs[0])

# 埋め込みモデル（google-genai 統合版）
embed_model = GoogleGenAIEmbedding(
    model_name=embedding_model,
    api_key=api_key
)

# LLM (Gemini via google-genai)
llm = GoogleGenAI(
    model=llm_model,
    api_key=api_key,
    temperature=1.0
)

# VectorStore(Index) 作成
index = VectorStoreIndex.from_documents(
    docs,
    embed_model=embed_model,
    storage_context=StorageContext.from_defaults()
)

# 確認:（オプション）retriever ------------------------
retriever = index.as_retriever(similarity_top_k=2)
test_query = '山田さんの職業は何ですか。'
retrieved = retriever.retrieve(test_query)
print(f"test vectorstore: Question={test_query}")
for i, node in enumerate(retrieved, start=1):
    print(f"--- retrieved {i} --- text={node.text}")
# --------------------------------------------------

# Query エンジンを構成
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=3
)

# Q&A 実行
queries = [
    '佐藤さんの職業は何ですか。',
    'お酒を飲む人は誰ですか。',
    '関東に住んでいる人は誰ですか。',
]
print("\nQ&A ----------------")
for i, query in enumerate(queries):
    resp = await query_engine.aquery(query) # 非同期実行
    print(f"\nQ{i} {query}; answer={resp.response}")
    for j, src in enumerate(resp.source_nodes, start=1):
        print(f"evidence{j} = {src.text}")


Doc ID: 0fce83b0-26d5-4d82-ae43-b8c5e6c76a0b
Text: 石川です。エンジニアです。散歩は嫌いではありません。
test vectorstore: Question=山田さんの職業は何ですか。
--- retrieved 1 --- text=果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
--- retrieved 2 --- text=佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。

Q&A ----------------

Q0 佐藤さんの職業は何ですか。; answer=佐藤さんはコンビニの店長です。

evidence1 = 佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。
evidence2 = 京都にお住いの小林さんは、佐藤さんのおいです。
evidence3 = 東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。

Q1 お酒を飲む人は誰ですか。; answer=渡辺さんと佐藤さんは居酒屋で飲みます。中村さんは高橋さんに飲みに誘われます。

evidence1 = 東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。
evidence2 = 果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
evidence3 = 中村さんは高橋さんの勤めている会社の上司です。よく飲みに誘われますが参加するかは半々です。

Q2 関東に住んでいる人は誰ですか。; answer=渡辺さん、佐藤さん、伊藤さん、山本さんは関東に住んでいるようです。

evidence1 = 東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。
evidence2 = 京都にお住いの小林さんは、佐藤さんのおいです。
evidence3 = 伊藤さんと山本さんは、よく一緒に奥多摩へキャンプに行くようです


## 大きなドキュメントの分割、メタデータの活用

ドキュメントとして、青空文庫(https://www.aozora.gr.jp/)を利用させていただきました。


In [3]:
import os
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor

metadata_dic = {
    '1_kumono_ito.txt': {
        'genre': 'novel', 'title':'蜘蛛の糸',          'author':'芥川竜之介', 'year':'1918'},
    '2_chumonno_oi_ryoriten.txt': {
        'genre': 'novel', 'title':'注文の多い料理店',  'author':'宮沢賢治',   'year':'1924'},
    '2_kazeno_matasaburo.txt': {
        'genre': 'novel', 'title':'風の又三郎',        'author':'宮沢賢治',   'year':'1934'},
    '2_serohikino_goshu.txt': {
        'genre': 'novel', 'title':'セロ弾きのゴーシュ', 'author':'宮沢賢治',   'year':'1934'},
    '2_gingatetsudono_yoru.txt': {
        'genre': 'novel', 'title':'銀河鉄道の夜',      'author':'宮沢賢治',   'year':'1934'},
    '3_bocchan.txt': {
        'genre': 'novel', 'title':'坊ちゃん',         'author':'夏目漱石',   'year':'1906'},
    '4_wagahaiwa_nekodearu.txt': {
        'genre': 'novel', 'title':'吾輩は猫である',    'author':'夏目漱石',   'year':'1905'},
    '4_kaijin_nijumenso.txt': {
        'genre': 'novel', 'title':'怪人二十面相',      'author':'江戸川乱歩', 'year':'1936'},
    '5_sanshodayu.txt': {
        'genre': 'novel', 'title':'山椒大夫',         'author':'森鷗外',     'year':'1915'},
}
null_dic = {'genre':'','title':'', 'author':'','year':''}

# 各テキストを読みながらドキュメント化しメタデータを付与
novels_dir = './novels/'
documents = []
for fn in os.listdir(novels_dir):
    if not fn.endswith('.txt'):
        continue
    file_path = os.path.join(novels_dir, fn)

    with open(file_path, 'r', encoding='utf-8') as f:  # ファイルを読み込む
        text = f.read()

    meta = metadata_dic.get(fn, null_dic)             # metadata を辞書から取得
    meta['file_name'] = fn                             # ファイル名も追加

    # Document 作成
    doc = Document(text=text)
    doc.metadata = meta   # ←ここで明示的にセット
    documents.append(doc)

print(documents[0].metadata, documents[0].text[:100]) # 確認

# ドキュメントをノード（チャンク）に分割
splitter = SentenceSplitter(    # 文単位で分割
    chunk_size=10000,
    chunk_overlap=100,
)
nodes = splitter.get_nodes_from_documents(documents)

# 確認: Node 内容-------------------------------------
print(len(nodes), type(nodes))
for i, n in enumerate(nodes[:2]):
    print(f"{'-'*10} \n{i}: {n.metadata}, \n{n.text[:200]}")
# --------------------------------------------------

{'genre': 'novel', 'title': '蜘蛛の糸', 'author': '芥川竜之介', 'year': '1918', 'file_name': '1_kumono_ito.txt'} 蜘蛛の糸
芥川龍之介

［＃８字下げ］一［＃「一」は中見出し］

　ある日の事でございます。御釈迦様《おしゃかさま》は極楽の蓮池《はすいけ》のふちを、独りでぶらぶら御歩きになっていらっしゃいました。池
42 <class 'list'>
---------- 
0: {'genre': 'novel', 'title': '蜘蛛の糸', 'author': '芥川竜之介', 'year': '1918', 'file_name': '1_kumono_ito.txt'}, 
蜘蛛の糸
芥川龍之介

［＃８字下げ］一［＃「一」は中見出し］

　ある日の事でございます。御釈迦様《おしゃかさま》は極楽の蓮池《はすいけ》のふちを、独りでぶらぶら御歩きになっていらっしゃいました。池の中に咲いている蓮《はす》の花は、みんな玉のようにまっ白で、そのまん中にある金色《きんいろ》の蕊《ずい》からは、何とも云えない好《よ》い匂《におい》が、絶間《たえま》なくあたりへ溢《あふ》れて居ります
---------- 
1: {'genre': 'novel', 'title': '注文の多い料理店', 'author': '宮沢賢治', 'year': '1924', 'file_name': '2_chumonno_oi_ryoriten.txt'}, 
注文の多い料理店
宮沢賢治


　二人の若い紳士《しんし》が、すっかりイギリスの兵隊のかたちをして、ぴかぴかする鉄砲《てっぽう》をかついで、白熊《しろくま》のような犬を二｜疋《ひき》つれて、だいぶ山奥《やまおく》の、木の葉のかさかさしたとこを、こんなことを云《い》いながら、あるいておりました。
「ぜんたい、ここらの山は怪《け》しからんね。鳥も獣《けもの》も一疋も居やがらん。なんでも構わないから、早


In [4]:
# 埋め込みモデル（google-genai 統合版）
embed_model = GoogleGenAIEmbedding(
    model_name=embedding_model,
    api_key=api_key
)

# VectorStore(Index) 作成
index = VectorStoreIndex(
    nodes,
    embed_model=embed_model,
    storage_context=StorageContext.from_defaults()
)

# 確認:（オプション）retriever ------------------------
retriever = index.as_retriever(similarity_top_k=2)
test_query = '御釈迦様はどこにいましたか。'
retrieved = retriever.retrieve(test_query)
print(f"test vectorstore: Question={test_query}")
for i, node in enumerate(retrieved, start=1):
    print(f"--- retrieved {i} --- {node.metadata.get('title', 'N/A')}\ntext={node.text[:200]}")
# --------------------------------------------------

test vectorstore: Question=御釈迦様はどこにいましたか。
--- retrieved 1 --- 蜘蛛の糸
text=蜘蛛の糸
芥川龍之介

［＃８字下げ］一［＃「一」は中見出し］

　ある日の事でございます。御釈迦様《おしゃかさま》は極楽の蓮池《はすいけ》のふちを、独りでぶらぶら御歩きになっていらっしゃいました。池の中に咲いている蓮《はす》の花は、みんな玉のようにまっ白で、そのまん中にある金色《きんいろ》の蕊《ずい》からは、何とも云えない好《よ》い匂《におい》が、絶間《たえま》なくあたりへ溢《あふ》れて居ります
--- retrieved 2 --- 山椒大夫
text=山椒大夫のところに来てから、二人一しょに歩くのはこれがはじめである。
　厨子王は姉の心を忖《はか》りかねて、寂しいような、悲しいような思いに胸が一ぱいになっている。きのうも奴頭の帰ったあとで、いろいろに詞を設けて尋ねたが、姉はひとりで何事をか考えているらしく、それをあからさまには打ち明けずにしまった。
　山の麓に来たとき、厨子王はこらえかねて言った。「姉えさん。わたしはこうして久しぶりで一しょに歩


In [5]:
import asyncio, time

# LLM (Gemini via google-genai)
llm = GoogleGenAI(
    model=llm_model,
    api_key=api_key,
    temperature=1.0
)

# Query エンジンの構成
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=3
)

# Q&A 実行
queries = [
    '蜘蛛の糸は何に使われましたか',
    'ゴーシュは何をしていますか',
    '坊ちゃんの姓名は何ですか',
    '怪人の名前は何ですか',
    '二十面相が現れた場所をすべて挙げてください',
    '猫の名は何ですか',
    '山椒大夫は最後にどうなりましたか',
]

print("\nQ&A ----------------")
max_retries = 3 # 最大リトライ回数
wait = 30       # リトライ待ち時間
timeout = 30    # 最大待ち時間(秒)

for i, query in enumerate(queries):
    for ir in range(1, max_retries+1): # 成功するまで繰り返し
        try:
            resp = await asyncio.wait_for(query_engine.aquery(query), timeout=timeout) # 非同期実行
            print(f"\nQ{i} {query}; answer={resp.response}")
            for i_src, src in enumerate(resp.source_nodes, start=1):
                print(f"evidence{i_src}, title:{src.metadata.get('title', 'N/A')} \n{src.text[:200]}")
            break                      # 成功したら脱出
        except asyncio.TimeoutError:
            print(f"TimeoutError: Retry {ir}/{max_retries} after {wait} secs")
        except Exception as e:
            print(e)
            print(f"Exceed request rate quota: Retry {ir}/{max_retries} after {wait} secs")
            await asyncio.sleep(wait)


Q&A ----------------

Q0 蜘蛛の糸は何に使われましたか; answer=蜘蛛の糸は、地獄にいる罪人を救い出すために使われました。

evidence1, title:蜘蛛の糸 
蜘蛛の糸
芥川龍之介

［＃８字下げ］一［＃「一」は中見出し］

　ある日の事でございます。御釈迦様《おしゃかさま》は極楽の蓮池《はすいけ》のふちを、独りでぶらぶら御歩きになっていらっしゃいました。池の中に咲いている蓮《はす》の花は、みんな玉のようにまっ白で、そのまん中にある金色《きんいろ》の蕊《ずい》からは、何とも云えない好《よ》い匂《におい》が、絶間《たえま》なくあたりへ溢《あふ》れて居ります
evidence2, title:怪人二十面相 
そこで、今まで寝ていた長イスを、窓の下へおしていって、それを踏み台に、のびあがってみましたが、それでもまだ窓へとどきません。子どもの力で重い長イスをたてにすることはできないし、ほかに踏み台にする道具とても見あたりません。
　では、小林君は、せっかく窓を発見しながら、そこから外をのぞくことも、できなかったのでしょうか。いやいや、読者諸君、ご心配にはおよびません。こういうときの用意に、なわばしごという
evidence3, title:怪人二十面相 
だが、どうもまだふ［＃「ふ」に傍点］におちねえことがある。さっき玄関へきたばっかりの時に、どうして、おかしらにあっしの姿が見えたんですかい。」
「ハハハ……、それかい。それはね。ほら、ここをのぞいてみたまえ。」
　首領は天井の一｜隅《ぐう》からさがっているストーブのえんとつみたいな物を指さしました。
　のぞいてみよといわれるものですから、赤井はそこへ行って、えんとつの下のはしがかぎの手に曲がってい

Q1 ゴーシュは何をしていますか; answer=ゴーシュは町の活動写真館でセロを弾く係りです。
evidence1, title:セロ弾きのゴーシュ 
セロ弾きのゴーシュ
宮沢賢治

　ゴーシュは町の活動写真館でセロを弾く係りでした。けれどもあんまり上手でないという評判でした。上手でないどころではなく実は仲間の楽手のなかではいちばん下手でしたから、いつでも楽長にいじめられるのでした。
　ひるすぎみんなは楽屋に円くならんで今度の町の音楽会へ出す第六｜交響曲《こうきょうきょく》の練習を